In [1]:
addprocs(4);

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
@everywhere module MNIST
using Knet,Compat,GZip

function predict(w,x)
    for i=1:2:length(w)
        x = w[i]*x .+ w[i+1]
        if i<length(w)-1
            x = relu(x) # max(0,x)
        end
    end
    return x
end

function loss(w,x,ygold)
    ypred = predict(w,x)
    ynorm = logp(ypred,1) # ypred .- log(sum(exp(ypred),1))
    -sum(ygold .* ynorm) / size(ygold,2)
end

lossgradient = grad(loss)

function train(w, dtrn; lr=.5, epochs=10)
    for epoch=1:epochs
        for (x,y) in dtrn
            g = lossgradient(w, x, y)
            for i in 1:length(w)
                w[i] -= lr * g[i]
            end
        end
    end
    return w
end

function err(w,dtrn)
    cost = 0.0
    for (x, ygold) in dtrn
        cost += loss(w,x,ygold)
    end
    cost
end

function accuracy(w, dtst)
    ncorrect = ninstance = 0
    for (x, ygold) in dtst
        ypred = predict(w, x)
        ncorrect += sum(ygold .* (ypred .== maximum(ypred,1)))
        ninstance += size(ygold,2)
    end
    return ncorrect/ninstance
end

function weights(h...; atype=Array{Float32}, winit=0.1)
    w = Any[]
    x = 28*28
    for y in [h..., 10]
        push!(w, convert(atype, winit*randn(y,x)))
        push!(w, convert(atype, zeros(y, 1)))
        x = y
    end
    return w
end

function minibatch(x, y, batchsize; atype=Array{Float32}, xrows=784, yrows=10, xscale=255)
    xbatch(a)=convert(atype, reshape(a./xscale, xrows, div(length(a),xrows)))
    ybatch(a)=(a[a.==0]=10; convert(atype, sparse(convert(Vector{Int},a),1:length(a),one(eltype(a)),yrows,length(a))))
    xcols = div(length(x),xrows)
    xcols == length(y) || throw(DimensionMismatch())
    data = Any[]
    for i=1:batchsize:xcols-batchsize+1
        j=i+batchsize-1
        push!(data, (xbatch(x[1+(i-1)*xrows:j*xrows]), ybatch(y[i:j])))
    end
    return data
end

function loaddata()
    info("Loading MNIST...")
    gzload("train-images-idx3-ubyte.gz")[17:end],
    gzload("t10k-images-idx3-ubyte.gz")[17:end],
    gzload("train-labels-idx1-ubyte.gz")[9:end],
    gzload("t10k-labels-idx1-ubyte.gz")[9:end]
end

function gzload(file; path=Pkg.dir("Knet","data",file), url="http://yann.lecun.com/exdb/mnist/$file")
    isfile(path) || download(url, path)
    f = gzopen(path)
    a = @compat read(f)
    close(f)
    return(a)
end

if !isdefined(:xtrn)
    (xtrn,xtst,ytrn,ytst)=loaddata()
end

function vector2weight(x)
    pos = 1
    for i in eachindex(w)
        n = length(w[i])
        w[i] = reshape(view(x,pos:pos+n-1),size(w[i]))
        pos += n
    end
end
    
function weight2vector(x)
    x = randn(sum(length.(w)),1)
end
    
function objective(x)
    vector2weight(x)
    # err(w,dtrn)
    Float64(1-accuracy(w,dtrn))
end
export objective

batchsize = 64
dtrn = minibatch(MNIST.xtrn, MNIST.ytrn, batchsize)
dtst = minibatch(MNIST.xtst, MNIST.ytst, batchsize)

w = weights(200)
x0 = weight2vector(w)
export x0
end # module

# @everywhere using MNIST

INFO: Knet using GPU 0
INFO: Knet using GPU 0
INFO: Knet using GPU 0
INFO: Knet using GPU 0
INFO: Knet using GPU 1
INFO: Knet using GPU 1
INFO: Knet using GPU 1
INFO: Knet using GPU 1
INFO: Knet using GPU 1
INFO: Loading MNIST...
INFO: Loading MNIST...
INFO: Loading MNIST...
INFO: Loading MNIST...
INFO: Loading MNIST...


In [ ]:
using BlackBoxOptim,MNIST
opt = bbsetup(objective; SearchRange = (-10.0, 10.0), 
NumDimensions = length(x0),PopulationSize = 10,
TraceMode = :silent, MaxSteps = 1,
Workers = workers());

res = bboptimize(opt)
for t=1:10
    res = bboptimize(opt; MaxSteps = 3)
    @show best_fitness(res)
end

In [ ]:
# name = tempname()
# open(name, "w") do fh
#     serialize(fh, opt)
# end
# open(name,"r") do fh
#     opt = deserialize(fh)
# end